## Daily price data from Diem pot delivery Portland website
Use requests, Beautiful Soup, and Pandas packages to retrieve html data from https://portland.hellodiem.com/, to locate product information, parse title and price into lists, remove superfluous pre-sale values, combine into a Pandas DataFrame, and save to a csv file.

In [1]:
def get_heading_list(url, page):
    # Packages the request, send the request and catch the response: r
    r = requests.get(url+'page/'+str(page)+'/')
    html_text = r.text

    #set soup list equal to html text from page
    soup = BeautifulSoup(html_text, 'html.parser')
    
    headings = [[], []]
    #get list of product title headings on page
    headings[0] = soup.find_all('h2', class_='woocommerce-loop-product__title')
    
    #get list of product price headings on page
    headings[1] = soup.find_all('span', class_='woocommerce-Price-amount amount')
    pop_extra_price(headings[1])
    return headings

def pop_extra_price(headingss):
    """helper function to remove extra pre-sale prices from final list. Input list of 
    headers, get back list of headers less those that had a particular tag"""
    for item in headingss:
        if item.parent.name == 'del':
            headingss.pop(headingss.index(item))


In [2]:
def strip_product(header_list):
    """input a list of tag-type values and return list of strings with surrounding html characters removed"""
    string_list = ['' for item in range(len(header_list))]
    for item in range(len(header_list)):
        string_list[item] = str(header_list[item])[44:-5]
    return string_list

def strip_price(header_list):
    """input a list of tag-type values and return list of strings with surrounding html characters removed"""
    string_list = ['' for item in range(len(header_list))]
    for item in range(len(header_list)):
        string_list[item] = str(header_list[item])[101:-7]
    return string_list

In [3]:
def build_df(price_sers):
    today_date = str(dt.date.today())
    
    combined = pd.concat(price_sers, axis=1)
    combined.columns = ['prices_' + str(i) for i in range(1, (len(combined.columns)) + 1)]
    combined.index.name = 'items'
    
    return combined

In [4]:
from bs4 import BeautifulSoup
import requests
max_page = 12
url = 'https://portland.hellodiem.com/'

In [5]:
#create a list to hold lists of headers, one for each page
h_product = ['' for strng in range(max_page)]
h_price = ['' for strng in range(max_page)]

#iterate over each list of headers for a page, set each index of headers equal to list
for page in range(0, max_page):
    headers = get_heading_list(url, page+1)
    h_product[page] = headers[0]
    h_price[page] = headers[1]
    
combined_products = []
combined_prices = []

for sublist in range(max_page):
    combined_products.append(strip_product(h_product[sublist]))
    
for sublist in range(max_page):
    combined_prices.append(strip_price(h_price[sublist]))

#create a list of all prices
all_prices = []
for by_page in combined_prices:
    for li in by_page:
        #print(li, '\n')
        all_prices.append(float(li))
        
#create a list of all products
all_products = []
for by_page in combined_products:
    for li in by_page:
        #print(li, '\n')
        all_products.append(li)

In [6]:
import pandas as pd
product_ser = pd.Series(all_products)
price_ser = pd.Series(all_prices)

pot = pd.DataFrame([product_ser, price_ser]).T

In [7]:
#import datetime, needed to get current date
import datetime as dt
#set today_date, formatted as YYYY-MM-DD
today_date = str(dt.date.today())
#print(today_date)

In [8]:
#------------------------------------------------------------------------------

#write to csv and set index to position rather than name, use
pot.to_csv(('prices'+ today_date +'.csv'), encoding='utf8', 
           header=['item_name', 'item_price'], index_label='relative_position')

#did not explicitly specify utf8 encoding when creating first file, and now need 
#to specify ansi when opening
pot1 = pd.read_csv('prices2018-05-07.csv', encoding='ansi')
pot1.set_index(pot1['0'], inplace=True)
pot1 = pot1[["1"]]
pot1.columns = ['item_price']
pot1.index.name = 'item_name'

#put in loop when more are available
#read files into dataframes to prepare for combination
pot2 = pd.read_csv('prices2018-05-08.csv', index_col='item_name')
pot2 = pot2[['item_price']]

pot3 = pd.read_csv('prices2018-05-09.csv', index_col='item_name')
pot3 = pot3[['item_price']]

In [9]:
pots = [pot1, pot2, pot3]
pot_all = build_df(pots)


In [10]:
#quick stats
print(pot_all.describe(), '\n')

#most expensive item and price
print(pot_all.idxmax()[0], ('$' + str(pot_all.loc[pot_all.idxmax()[0]][0])))

         prices_1    prices_2    prices_3
count  211.000000  210.000000  210.000000
mean    38.338863   38.416667   38.416667
std     33.591549   33.652753   33.652753
min      1.750000    1.750000    1.750000
25%     13.000000   13.000000   13.000000
50%     27.000000   27.000000   27.000000
75%     54.000000   54.000000   54.000000
max    200.000000  200.000000  200.000000 

DISCOUNT OUNCE ~ Mendo Breath ~ ($164 OFF) $200.0


In [11]:
#check which items have changed price, sold out, or been added
for i in range(len(pot_all)):
    p = 0
    if (pot_all.iloc[i][p] != pot_all.iloc[i][p+1]):
        print(pot_all.iloc[i])
    if (pot_all.iloc[i][p+1] != pot_all.iloc[i][p+2]):
        print(pot_all.iloc[i])

prices_1    40.0
prices_2    33.0
prices_3    33.0
Name: Blue Dragon Oil Cartridge 1/2 gram, dtype: float64
prices_1    33.0
prices_2    40.0
prices_3    40.0
Name: Blue Mystic Oil Cartridge 1 gram, dtype: float64
prices_1    22.0
prices_2     NaN
prices_3     NaN
Name: CBD Pre-roll Sampler 3-pack ($5 OFF), dtype: float64
prices_1    22.0
prices_2     NaN
prices_3     NaN
Name: CBD Pre-roll Sampler 3-pack ($5 OFF), dtype: float64
prices_1    42.0
prices_2    40.0
prices_3    40.0
Name: Fruit Punch Oil Cartridge 1/2 gram, dtype: float64
prices_1    40.0
prices_2    42.0
prices_3    42.0
Name: Jilly Bean Live Resin Cartridge 1/2 gram, dtype: float64
